### Import a pretrained model

Load vgg19 architecture, with its <b> pretrained weights </b> using torchvision's models module. These are the weights coming from the training using ImageNet dataset with 1000 classes like in LSVRC competition. There are various models provided in https://pytorch.org/vision/0.8/models.html with pretrained weights, I suggest you to take a look!

In [5]:
from torchvision import models
import torch
from torchsummary import summary

model = models.vgg19(pretrained=True)
            
summary(model, input_size = (3, 224, 224), batch_size = 1)



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 224, 224]           1,792
              ReLU-2          [1, 64, 224, 224]               0
            Conv2d-3          [1, 64, 224, 224]          36,928
              ReLU-4          [1, 64, 224, 224]               0
         MaxPool2d-5          [1, 64, 112, 112]               0
            Conv2d-6         [1, 128, 112, 112]          73,856
              ReLU-7         [1, 128, 112, 112]               0
            Conv2d-8         [1, 128, 112, 112]         147,584
              ReLU-9         [1, 128, 112, 112]               0
        MaxPool2d-10           [1, 128, 56, 56]               0
           Conv2d-11           [1, 256, 56, 56]         295,168
             ReLU-12           [1, 256, 56, 56]               0
           Conv2d-13           [1, 256, 56, 56]         590,080
             ReLU-14           [1, 256,

### Freeze the layers


As mentioned, you can keep these pretrained weights without updating them during your own train, or you may prefer to include them also to your back propagation process. Below code freeze the pretrained layers, so we choose not to update these weights during training. If you change as ``` param.requires_grad = True ``` <u> you declare that you want gradient calculation for these layers </u>, these layers will be included to the backpropagation also. When we print the model summary after freezing the layers, we see that now all the parameters we have are in <b> non-trainable params </b> category

In [6]:
for param in model.parameters():
   param.requires_grad = False

summary(model, input_size = (3, 224, 224), batch_size = 1)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 224, 224]           1,792
              ReLU-2          [1, 64, 224, 224]               0
            Conv2d-3          [1, 64, 224, 224]          36,928
              ReLU-4          [1, 64, 224, 224]               0
         MaxPool2d-5          [1, 64, 112, 112]               0
            Conv2d-6         [1, 128, 112, 112]          73,856
              ReLU-7         [1, 128, 112, 112]               0
            Conv2d-8         [1, 128, 112, 112]         147,584
              ReLU-9         [1, 128, 112, 112]               0
        MaxPool2d-10           [1, 128, 56, 56]               0
           Conv2d-11           [1, 256, 56, 56]         295,168
             ReLU-12           [1, 256, 56, 56]               0
           Conv2d-13           [1, 256, 56, 56]         590,080
             ReLU-14           [1, 256,

### Modify Last Layer

Now you have a prepared model for 1000 classes with its weights, but you need a model for 3 classes. So you have to update your output layer right? We can simply remove the last layer containing 1000 nodes and add a new one with 3 classes. Another approach would be not to 

In [7]:
print(model.classifier) # check model HEAD

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [8]:
nb_classes = 3

number_features = model.classifier[-1].in_features  # obtain input node amount for the new layer
features = list(model.classifier.children())[:-1]   # Remove last layer by putting -1 to range index, 
                                                    # So we obtain the HEAD layers' list witohut the last layer
features.extend([torch.nn.Linear(number_features, nb_classes)]) # We extend this list with the output layer we want to add
model.classifier = torch.nn.Sequential(*features) # replace new classifier HEAD with the old one

print(model.classifier) 

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=3, bias=True)
)
